# 49. Разработка чат-бота

Задачу нужно решить во время тестирования
Разработайте чат-бот, который в диалоговом режиме помогает абитуриенту разобраться, какая из двух
магистерских программ ему подходит и как наилучшим образом спланировать учёбу, исходя из учебных
планов, загруженных на страницах магистратур под запрос абитуриента:
- https://abit.itmo.ru/program/master/ai
- https://abit.itmo.ru/program/master/ai_product
1. Реализуйте парсинг данных с сайтов и соберите учебные планы.
2. Реализуйте диалоговую систему с ответами на вопросы по содержимому сайтов, например на базе
telegram-бота.
3. Реализуйте рекомендации - какие выборные дисциплины лучше прослушать абитуриенту в ходе
обучения, с учетом вводных от абитуриента о его бэкграунде.
4. Важно, чтобы чат-бот отвечал отвечал только на релевантные вопросы по обучению в выбранных магистратурах.

Пользуйтесь любым удобным инструментарием при решении задачи и опишите, как решали задачу, чем пользовались и как принимали решения. Добавьте в поле ссылку на открытый репозиторий или выложенный архив с решением.
Система Контеста выдаст ошибку на ссылку - это сообщение можно игнорировать, решение будет зачтено.

# Parsing

In [40]:
import pandas as pd
import os
import time
from datetime import datetime as dt

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import PyPDF2
import camelot

from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from transformers import TapasTokenizer, TapasForQuestionAnswering
import torch

import telebot

import warnings
warnings.filterwarnings('ignore')

In [30]:
DRIVER_PATH = os.path.join('C:\\','chromedriver-win64_139','chromedriver.exe')

urls = ['https://abit.itmo.ru/program/master/ai',
        'https://abit.itmo.ru/program/master/ai_product']

DOWNLOAD_DIRR = os.path.join('C:\\','Users','Admin','Downloads')
# название скачанный файлов с учебными программами
pdf_ai = '10033-abit'
pdf_ai_product = '10130-abit'

In [24]:
# скачивание пдф с учебный планом
def download_plan(url, elem):

    options = webdriver.ChromeOptions()
    service = Service(executable_path=DRIVER_PATH)
    driver = webdriver.Chrome(service=service, options=options)
    driver.maximize_window()

    driver.get(url)
    driver.find_element(By.CSS_SELECTOR,elem).click()
    time.sleep(2)
    driver.close()
    
# скачиваем учебный план    
# ИИ
download_plan(urls[0], elem = '#__next > div > main > div.Background_background__im0jM.full-width.contain.flow.Background_purple__FnYHm > div.StudyPlan_plan__aIqUC > div > div > button')
# ИИ в продукте
download_plan(urls[1], elem = '#__next > div > main > div.Background_background__im0jM.full-width.contain.flow.Background_purple__FnYHm > div.StudyPlan_plan__aIqUC > div > div > button')

In [92]:
# все скачанные учебные планы 
curriculum_pdf = [i for i in os.listdir(DOWNLOAD_DIRR) if pdf_ai in i or pdf_ai_product in i] 
# нужен последний скачанный файл (время создания файла макс)
pdf_dict = {x: dt.fromtimestamp(os.path.getmtime(os.path.join(DOWNLOAD_DIRR, x))) for x in curriculum_pdf}
# сортируем по времени скачивания
sorted_files = dict(sorted(
    pdf_dict.items(),
    key=lambda x: x[1],
    reverse=True
))
# название двух последних скачанных файлов
ai_product, ai = list(sorted_files.keys())[:2]; ai_product, ai

('10130-abit (1).pdf', '10033-abit (6).pdf')

In [97]:
def read_pdf(file):
    tables = camelot.read_pdf(
        os.path.join(DOWNLOAD_DIRR, file),
        flavor='lattice',  # 'stream' для таблиц без линий
        pages='all'
    )
    curriculum = pd.DataFrame()
    for i, table in enumerate(tables):
        curriculum = pd.concat([curriculum,table.df], axis = 0) 
    return curriculum

# таблицы с учебными планами
# ИИ
curriculum_ai = read_pdf(ai)
# ИИ в продукте
curriculum_ai_product = read_pdf(ai_product)

In [100]:
curriculum_ai.to_excel('Учебный план_ИИ.xlsx', index = False)
curriculum_ai_product.to_excel('Учебный план_ИИ_product.xlsx', index = False)

<ipython-input-100-c068bb5a50d1>:1: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  curriculum_ai.to_excel('Учебный план_ИИ.xlsx', index = False)
<ipython-input-100-c068bb5a50d1>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  curriculum_ai_product.to_excel('Учебный план_ИИ_product.xlsx', index = False)


# telegram-bot

In [ ]:
TOKEN = ""
bot = telebot.TeleBot(TOKEN)

curriculum_ai = pd.read_excel('Учебный план_ИИ.xlsx')
curriculum_ai_product = pd.read_excel('Учебный план_ИИ_product.xlsx')

curriculum_ai.drop_duplicates(inplace = True)
curriculum_ai_product.drop_duplicates(inplace = True)

# Формируем контекст
context = f'''Учебный план программы Искусственный интеллект ИТМО: {str(curriculum_ai)}
Учебный план программы Искусственный интеллект в продукте ИТМО: {str(curriculum_ai_product)}'''


def answer_question_LLM(context: str, question: str) -> str:

    # Используем рабочую модель для русского языка
    model_name = "AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru"
    
    # Создаем пайплайн для вопросно-ответных задач
    nlp = pipeline(
        'question-answering', 
        model=model_name,
        tokenizer=model_name,
        device=0 if torch.cuda.is_available() else -1
    )
    
    # Формируем входные данные
    inputs = {
        'question': question,
        'context': context
    }
    
    # Получаем ответ
    try:
        result = nlp(inputs)
        return result['answer'] if result['score'] > 0.01 else "Ответ не найден в контексте"
    except Exception as e:
        return f"Ошибка обработки: {str(e)}"



# Обработка сообщений
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    user_question = message.text
    answer = answer_question_LLM(context, user_question)
    bot.reply_to(message, answer)

print("Бот запущен...")
bot.infinity_polling()